<a href="https://colab.research.google.com/github/illusoryTwin/FoR/blob/mass_matrix/assig3/for_assig3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define the fundamental functions

### Transformation matrices

In [1]:
import numpy as np

# Define rotation and translation functions
def rotate_z(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                     [np.sin(theta), np.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def rotate_x(theta):
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

def rotate_y(theta):
    return np.array([[np.cos(theta), 0, np.sin(theta), 0],
                     [0, 1, 0, 0],
                     [-np.sin(theta), 0, np.cos(theta), 0],
                     [0, 0, 0, 1]])

def translate_x(d):
    return np.array([[1, 0, 0, d],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def translate_y(d):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, d],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

def translate_z(d):
    return np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, d],
                     [0, 0, 0, 1]])


def validate_matrices(matrices):
  tolerance = 1e-12

  # Define a function to check if a value is close to zero within the tolerance
  def is_close_to_zero(value):
      return abs(value) < tolerance

  validated_matrices = []

  for matrix in matrices:
    for i in range(matrix.shape[0]):
      for j in range(matrix.shape[1]):
        if is_close_to_zero(matrix[i][j]):
          matrix[i][j] = 0
    validated_matrices.append(matrix)

  return validated_matrices



### Symbolic transformation matrices

In [2]:
import sympy as sp

# Define symbolic rotation and translation functions
def rotate_z_sym(theta):
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta), 0, 0],
        [sp.sin(theta), sp.cos(theta), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def rotate_x_sym(theta):
    return sp.Matrix([
        [1, 0, 0, 0],
        [0, sp.cos(theta), -sp.sin(theta), 0],
        [0, sp.sin(theta), sp.cos(theta), 0],
        [0, 0, 0, 1]
    ])

def rotate_y_sym(theta):
    return sp.Matrix([
        [sp.cos(theta), 0, sp.sin(theta), 0],
        [0, 1, 0, 0],
        [-sp.sin(theta), 0, sp.cos(theta), 0],
        [0, 0, 0, 1]
    ])

def translate_x_sym(d):
    return sp.Matrix([
        [1, 0, 0, d],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def translate_y_sym(d):
    return sp.Matrix([
        [1, 0, 0, 0],
        [0, 1, 0, d],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def translate_z_sym(d):
    return sp.Matrix([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, d],
        [0, 0, 0, 1]
    ])

def validate_matrix_sym(matrix):
    tolerance = 1e-12
    # Define a function to check if a value is close to zero within the tolerance
    def is_close_to_zero(value):
        return abs(value) < tolerance

    # Iterate through each element in the matrix
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            # If the element is close to zero, set it to zero
            if is_close_to_zero(matrix[i, j]):
                matrix[i, j] = 0


# Solving dynamics
### Define the Manipulator class

In [3]:
import numpy as np

class Manipulator:
    def __init__(self, l, m=[1 for _ in range(6)], r=[0.05 for _ in range(6)]):
        self.l = l  # length of the links
        self.m = m  # mass of the links
        self.q = q  # configuration parameters (angles)
        self.r = r  # radii of cylindrical links
        self.h = l  # heights of cylindrical links
        self.lc = [l_i / 2 for l_i in l]  # distances to the links' CoMs

    def set_config(self, q):
        self.q = q

    def get_config(self):
        return self.q

    def get_config_in_pi(self):
        return [str(q_ / np.pi) + "pi" for q_ in self.q]


l = [1 for _ in range(6)]
m = [1 for _ in range(6)]
q = [np.pi/2, np.pi/3, np.pi/4, 0, np.pi/12, 0]

manipulator = Manipulator(l, m)
manipulator.set_config(q)

# Get the configuration parameters in terms of pi
config_in_pi = manipulator.get_config_in_pi()
print(config_in_pi)


['0.5pi', '0.3333333333333333pi', '0.25pi', '0.0pi', '0.08333333333333333pi', '0.0pi']


### Find center of mass

In [4]:
def calculate_com(Manipiulator: manipulator):
    theta1, theta2, theta3, theta4, theta5, theta6 = manipulator.get_config()
    lc = manipulator.lc

    T_01 = rotate_z(theta1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
    com1 = T_01 @ translate_z(lc[0])

    T_02 = T_01 @ translate_z(l[0]) @ rotate_z(theta2)
    com2 = T_02 @ translate_x(lc[1])

    T_03 = T_02 @ translate_x(l[1]) @ rotate_z(theta3) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)
    com3 = T_03 @ translate_x(lc[2])

    T_04 = T_03 @ translate_x(l[2]) @ rotate_z(theta4) @ rotate_x(-np.pi/2) @ rotate_z(-np.pi)
    com4 = T_04 @ translate_z(lc[3])

    T_05 = T_04 @ translate_z(l[3]) @ rotate_z(theta5) @ rotate_x(-np.pi/2)
    com5 = T_05 @ translate_y(lc[4])

    T_06 = T_05 @ translate_y(l[4]) @ rotate_z(theta6)
    com6 = T_06 @ translate_z(lc[5])
    coms = [com1, com2, com3, com4, com5, com6]
    coms = validate_matrices(coms)
    return coms


coms_matrices = calculate_com(manipulator)
coms_matrices

[array([[-1. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. ,  0.5],
        [ 0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  1. ]]),
 array([[-0.5      ,  0.8660254,  0.       , -0.25     ],
        [ 0.       ,  0.       ,  1.       ,  1.       ],
        [ 0.8660254,  0.5      ,  0.       ,  0.4330127],
        [ 0.       ,  0.       ,  0.       ,  1.       ]]),
 array([[ 0.96592583,  0.        ,  0.25881905, -0.01703709],
        [ 0.        ,  1.        ,  0.        ,  1.        ],
        [-0.25881905,  0.        ,  0.96592583,  0.73661588],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[-0.96592583,  0.25881905,  0.        ,  0.46592583],
        [ 0.        ,  0.        ,  1.        ,  1.5       ],
        [ 0.25881905,  0.96592583,  0.        ,  0.60720636],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[-0.8660254 ,  0.        ,  0.5       ,  0.46592583],
        [ 0.        , -1.        ,  0.        ,  1.5       ],
    

In [ ]:
from sympy import symbols

def calculate_com_symb(manipulator: Manipulator):
    theta1, theta2, theta3, theta4, theta5, theta6 = symbols('theta1 theta2 theta3 theta4 theta5 theta6')
    l = manipulator.l
    lc = manipulator.lc

    T_01 = rotate_z_sym(theta1) @ rotate_z(np.pi/2) @ rotate_x(np.pi/2)
    com1 = T_01 @ translate_z(lc[0])

    T_02 = T_01 @ translate_z(l[0]) @ rotate_z_sym(theta2)
    com2 = T_02 @ translate_x(lc[1])

    T_03 = T_02 @ translate_x(l[1]) @ rotate_z_sym(theta3) @ rotate_y(np.pi/2) @ rotate_z(np.pi/2)
    com3 = T_03 @ translate_x(lc[2])

    T_04 = T_03 @ translate_x(l[2]) @ rotate_z_sym(theta4) @ rotate_x(-np.pi/2) @ rotate_z(-np.pi)
    com4 = T_04 @ translate_z(lc[3])

    T_05 = T_04 @ translate_z(l[3]) @ rotate_z_sym(theta5) @ rotate_x(-np.pi/2)
    com5 = T_05 @ translate_y(lc[4])

    T_06 = T_05 @ translate_y(l[4]) @ rotate_z_sym(theta6)
    com6 = T_06 @ translate_z(lc[5])
    coms = [com1, com2, com3, com4, com5, com6]

    return coms

coms_matrices_symb = calculate_com_symb(manipulator)
coms_matrices_symb

In [6]:
# Let's assume that the manipulator's links are cylindrical

class InertiaMatrixForCylinder:
    def __init__(self, m, r, h):
        self.m = m
        self.r = r
        self.h = h

    def get_inertia_matrix(self):
        I = np.array([[self.m*self.h**2/2 + self.m*self.r**2/4, 0, 0],
                      [0, self.m*self.h**2/12 + self.m*self.r**2/4, 0],
                      [0, 0, self.m*self.r**2/2]])
        return I

In [10]:
# Example of usage

l = [1 for _ in range(6)]
m = [1 for _ in range(6)]
# q = [np.pi/2, np.pi/3, np.pi/4, 0, np.pi/12, 0]

manipulator = Manipulator(l, m)
# manipulator.set_config(q)

inertia_matrices = []


for i in range(6):
  inertia_matrix = InertiaMatrixForCylinder(manipulator.m[i], manipulator.r[i], manipulator.h[i])
  inertia_matrices.append(inertia_matrix.get_inertia_matrix())

inertia_matrices

[array([[0.500625  , 0.        , 0.        ],
        [0.        , 0.08395833, 0.        ],
        [0.        , 0.        , 0.00125   ]]),
 array([[0.500625  , 0.        , 0.        ],
        [0.        , 0.08395833, 0.        ],
        [0.        , 0.        , 0.00125   ]]),
 array([[0.500625  , 0.        , 0.        ],
        [0.        , 0.08395833, 0.        ],
        [0.        , 0.        , 0.00125   ]]),
 array([[0.500625  , 0.        , 0.        ],
        [0.        , 0.08395833, 0.        ],
        [0.        , 0.        , 0.00125   ]]),
 array([[0.500625  , 0.        , 0.        ],
        [0.        , 0.08395833, 0.        ],
        [0.        , 0.        , 0.00125   ]]),
 array([[0.500625  , 0.        , 0.        ],
        [0.        , 0.08395833, 0.        ],
        [0.        , 0.        , 0.00125   ]])]

In [17]:
# CHECK

# q1_symb, q2_symb, q3_symb, q4_symb, q5_symb, q6_symb = sp.symbols('theta1 theta2 theta3 theta4 theta5 theta6')
# q_symb = [q1_symb, q2_symb, q3_symb, q4_symb, q5_symb, q6_symb]

# print(sp.diff(coms_matrices_symb[0][:3, 3], q_symb[0]))
# coms_matrices_symb[0][:3, 3]

Matrix([[-0.5*sin(theta1) + 3.06161699786838e-17*cos(theta1)], [3.06161699786838e-17*sin(theta1) + 0.5*cos(theta1)], [0]])


Matrix([
[3.06161699786838e-17*sin(theta1) + 0.5*cos(theta1)],
[0.5*sin(theta1) - 3.06161699786838e-17*cos(theta1)],
[                              3.06161699786838e-17]])

In [31]:
q1_symb, q2_symb, q3_symb, q4_symb, q5_symb, q6_symb = sp.symbols('theta1 theta2 theta3 theta4 theta5 theta6')
q_symb = [q1_symb, q2_symb, q3_symb, q4_symb, q5_symb, q6_symb]

# def calculate_jacobian_v(com_symb):
#     jacobian_v = sp.zeros(6, 3, 6)

#     for k in range(6):
#         for i in range(3):
#             for j in range(6):
#                 jacobian_v[k, i, j] = sp.diff(com_symb[k][i, 3], q_symb[j])

#     return jacobian_v

def calculate_jacobian_v(com_symb):
    jacobian_v = []

    for k in range(6):
        row = []
        for i in range(3):
            col = []
            for j in range(6):
                col.append(sp.diff(com_symb[k][i, 3], q_symb[j]))
            row.append(col)
        jacobian_v.append(row)

    return sp.Array(jacobian_v)


# def calculate_jacobian_v(com_symb):
#     jacobian_v = sp.Matrix()

#     for k in range(6):
#         row = []
#         for i in range(3):
#             col = []
#             for j in range(6):
#                 col.append(sp.diff(com_symb[k][i, 3], q_symb[j]))
#             row.append(col)
#         jacobian_v.append(row)

#     # Convert the list of lists into a SymPy matrix
#     # jacobian_v = sp.Matrix(jacobian_v)
#     return jacobian_v

jac_v = calculate_jacobian_v(coms_matrices_symb)
jac_v = jac_v.subs({q1_symb: np.pi/12, q2_symb: np.pi/12, q3_symb: np.pi/12, q1_symb: np.pi/12, q1_symb: np.pi/12})
jac_v


[[[-0.12940952255126, 0, 0, 0, 0, 0], [0.482962913144534, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[-0.72532539604863, 0.0334936490538903, 0, 0, 0, 0], [0.840925826289068, -0.125, 0, 0, 0, 0], [0, 0.482962913144534, 0, 0, 0, 0]], [[-0.950350290422473, 0.179059232128787, 0.112071934021007, 0, 0, 0], [0.780630587564699, -0.668258151868904, -0.418258151868904, 0, 0, 0], [0, 0.715925826289068, -0.25, 0, 0, 0]], [[-0.241481456572267*sin(theta4) - 0.12940952255126*cos(theta4) - 0.708868833850206, -0.112071934021007*sin(theta4) + 2.90040282436426e-18*cos(theta4) + 0.291131166149794, -0.112071934021007*sin(theta4) + 2.90040282436426e-18*cos(theta4) + 0.224143868042013, -0.482962913144534*sin(theta4) - 0.0647047612756301*cos(theta4), 0, 0], [-0.0647047612756301*sin(theta4) + 0.482962913144534*cos(theta4) + 0.845335348840328, 0.418258151868904*sin(theta4) - 1.08244507029437e-17*cos(theta4) - 1.08651630373781, 0.418258151868904*sin(theta4) - 1.08244507029437e-17*cos(theta4) - 0.836516303737808, -0.1

In [ ]:
def calculate_jacobian_w(coms):
    jacobian_w = np.zeros((6, 3, 6))
    u = [np.array([0, 0, 1]),  coms[0][:3, 2],  coms[1][:3, 2], coms[2][:3, 2],  coms[3][:3, 2],  coms[4][:3, 2]]
    # print("u", u)
    # u = np.array([[0, 0, 1], coms[0, :3, 2], coms[1, :3, 2], coms[2, :3, 2], coms[3, :3, 2], coms[4, :3, 2]])  # Enclosed the array elements in double square brackets
    for i in range(6):
      for j in range(i+1):
        # print("u[j]", u[j])
        jacobian_w[i][:, j] = u[j]  # Corrected the assignment
    return jacobian_w

coms_matrices = calculate_com(manipulator)
jac_w = calculate_jacobian_w(coms_matrices)
print("jac", jac_w)